In [1]:
from __future__ import print_function
import tweepy
import csv, time

In [2]:
#Twitter API credentials
consumer_key = "unvLBd6FQnINy5gSOcZt28pld"
consumer_secret = "yMx5EXwKgI4KT6a6tsW7UG58Pem58RrGtI1NuufRaQqGNf8kTq"
access_key = "931572234-fQsUK0bpUW1qZv4j0EPI2WICxF7ftvtCLe3h1QK9"
access_secret = "mijow4MO0KDAF1Dst9AkJPRcCx9qEABXPXpmCAKJPurFH"


def get_all_tweets(user_id, folder = './'):
    # also returns number of api calls used
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(user_id = user_id,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    num_api_calls = 0
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        num_api_calls += 1
#         print("getting tweets before %s" % (oldest))
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(user_id = user_id,count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
#         print("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    
    #write the csv  
    with open('%s/%s_tweets.csv' % (folder, user_id), 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    
    return num_api_calls

In [3]:
def get_users(filename, folder = './'):
    f = open(filename, "r")
    total = 0
    start_time = time.time()
    for line in f:
        print(int(line))
        elapsed_time = time.time() - start_time
        if (elapsed_time - start_time) / 60 / 15 < 15 and total > 160:
            wait_time = 15*60 - ((elapsed_time - start_time) / 60)
            print('Sleeping for %.2f seconds'%wait_time)
            time.sleep(round(wait_time) + 1)
            print('Resetting timer and total api calls count')
            start_time = time.time()
            total = 0
        
        calls = get_all_tweets(int(line), folder)
        total += calls
        print("Made", calls, "API calls --", total, "total")

In [4]:
get_users('./d1.txt', folder = './dem_tweets')

813286
Made 17 API calls -- 17 total
939091
Made 6 API calls -- 23 total
980611
Made 17 API calls -- 40 total
1106501
Made 17 API calls -- 57 total
1947301
Made 17 API calls -- 74 total
3497941
Made 17 API calls -- 91 total
3819931
Made 17 API calls -- 108 total
4559531


RateLimitError: [{u'message': u'Rate limit exceeded', u'code': 88}]